In [12]:
#import import_ipynb
#from model import *
import numpy as np
import itertools
import glob
import os
import json
import re
import pickle
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.backend import mean, sum
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Concatenate, Multiply, Reshape
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.layers import Lambda
from keras.optimizers import Adam
np.random.seed(1)
from keras.models import Sequential


In [13]:
# given x (?,dim) always return (max_len, dim) by padding with zeros
def padding(max_len, x, dim ):
    x_len = x.shape[0]
    # truncate if more than max_len else pad with zeros
    if(x_len >= max_len):
        return x[0:max_len]
    else:   
        padding = np.zeros((max_len-x_len, dim))     
        if(len(x.shape) > 1):
            assert dim == x.shape[1],"dimensions didnt match in padding"   
            return np.concatenate((x, padding))
        else:
            return padding
        
# given 1D array x, we return array of shape (max_len, dim) by replicating each element dim times and apdding with zeros at end
def reshape_mask(max_len, x, dim):
    # mask is a 1D array so padd with dim == 1
    padded_question_mask = np.squeeze(padding(max_len, x, 1))
    final_question_mask = []
    for qm in padded_question_mask:
        # make each mask index into array of dimension dim
        final_question_mask.append(np.array([qm]*dim))
    return np.asarray(final_question_mask)


In [14]:
s_DimOfQuestionVector = 300
s_DimOfTripletVector  = 900
s_DimOfMask           = 50
s_QuestionMaxWords    = 60
s_TripletsMaxNumbers  = 500

In [15]:
def prepare_data(data, 
                 question_max_words, 
                 triplets_max_numbers, 
                 dimOfQuestionVector, 
                 dimOfTripletVector, 
                 dimOfMask):
    question_vectors = []
    question_masks = []
    triplet_vectors = []
    Y = []
    for elem in data:
        item_question             = elem["question"]
        padded_item_question      = padding(question_max_words, item_question, dimOfQuestionVector)
        item_question_mask        = np.reshape(elem["question_mask"], (len(elem["question_mask"]), 1) )
        padded_item_question_mask = reshape_mask(question_max_words, item_question_mask, dimOfMask)
        item_correct_triplets     = elem["correct_triplets"]
        item_wrong_triplets       = elem["wrong_triplets"]
        
        # for triplets take mean of each S, P  and 0 to get 3,300 vectors which are then concatenated to dimOfTripletVector vector
        item_correct_triplets_vectors = [] 
        for tr in item_correct_triplets: # There are triplets_max_numbers triplets
            # Each tr has s, p and o. Each of the _s, _p and _o's are (num_tokens, dimOfTripletVector) size
            _s = np.mean(tr[0], axis=0)
            _p = np.mean(tr[1], axis=0)
            _o = np.mean(tr[2], axis=0)
            item_correct_triplets_vectors.append(np.concatenate((_s, _p, _o))) # adding (900,) vector for each triplet
        #pad triplet vectors so we have (triplets_max_numbers, dimOfTripletVector)  encoding for all triplets
        padded_item_triplets_vectors = padding(triplets_max_numbers, np.asarray(item_correct_triplets_vectors), dimOfTripletVector)
        
        triplet_vectors.append(padded_item_triplets_vectors)
        question_vectors.append(padded_item_question)
        question_masks.append(padded_item_question_mask)
        Y.append(1)

        # for wrong triplets
        item_wrong_triplets_vectors = []
        for tr in item_wrong_triplets:
            _s = np.mean(tr[0], axis=0)
            _p = np.mean(tr[1], axis=0)
            _o = np.mean(tr[2], axis=0)
            item_wrong_triplets_vectors.append(np.concatenate((_s, _p, _o)))
        padded_item_wrong_triplets_vectors = padding(triplets_max_numbers, np.asarray(item_wrong_triplets_vectors), dimOfTripletVector)
                
        triplet_vectors.append(padded_item_wrong_triplets_vectors)
        question_vectors.append(padded_item_question)
        question_masks.append(padded_item_question_mask)
        Y.append(0)
        
        
        
    question_vectors = np.asarray(question_vectors)
    question_masks   = np.asarray(question_masks)
    triplet_vectors  = np.asarray(triplet_vectors)
    Y                = np.asarray(Y)
    return question_vectors, question_masks, triplet_vectors, Y

def generate_model_inputs():
    file_list = glob.glob('X:/cs230/proj0/ned-graphs/data_wiki_encoded/train/train_8.ipwk')
    data = []
    num = 0
    for inputFile in file_list:
        print(inputFile)
        f = open(inputFile, 'rb')
        test = pickle.load(f)
        data = np.append(data, test)
        f.close()
        num += 1
        if num == 1:
            break
    
    return prepare_data(data, 
                        question_max_words   = s_QuestionMaxWords, 
                        triplets_max_numbers = s_TripletsMaxNumbers, 
                        dimOfQuestionVector  = s_DimOfQuestionVector, 
                        dimOfTripletVector   = s_DimOfTripletVector, 
                        dimOfMask            = s_DimOfMask )

question_vectors, question_masks, triplet_vectors, Y = generate_model_inputs()#prepare_data(data, question_max_words=60, triplets_max_numbers=500)

X:/cs230/proj0/ned-graphs/data_wiki_encoded/train/train_8.ipwk


In [16]:
print(question_vectors.shape)
print(question_masks.shape)
print(triplet_vectors.shape)
print(Y.shape)

(200, 60, 300)
(200, 60, 50)
(200, 500, 900)
(200,)


In [17]:
questions_input_shape      = (s_QuestionMaxWords,   s_DimOfQuestionVector)
questions_mask_input_shape = (s_QuestionMaxWords,   s_DimOfMask )
triplets_input_shape       = (s_TripletsMaxNumbers, s_DimOfTripletVector )
# question_vectors(, 60, 300), question_masks(, 60, 300), triplet_vectors (,500,900), Y

gru_dimension = s_DimOfMask

In [18]:
# question inputs
question_vectors_input = Input(shape=questions_input_shape,      dtype='float32', name="question_vectors_input")   
question_masks_input   = Input(shape=questions_mask_input_shape, dtype='float32', name="question_masks_input")

# triplets input
triplets_input = Input(shape=triplets_input_shape, dtype='float32', name="triplets_input")  #(None, 500, 900)

# layers
# gru1 = Bidirectional(GRU( 50, return_sequences=True, name="Sentence_GRU1"))
# gru2 = Bidirectional(GRU( 50, return_sequences=False, name="Sentence_GRU2"))
# tgru1 = Bidirectional(GRU( 50, return_sequences=True, name="Triplet_GRU1"))
# tgru2 = Bidirectional(GRU( 50, return_sequences=False, name="Triplet_GRU2"))

gru1 =  (GRU( gru_dimension,  return_sequences=True,  name="Sentence_GRU1"))
gru2 =  (GRU( gru_dimension,  return_sequences=True,  name="Sentence_GRU2"))
tgru1 = (GRU( gru_dimension,  return_sequences=True,  name="Triplet_GRU1"))
tgru2 = (GRU( gru_dimension,  return_sequences=True, name="Triplet_GRU2"))
mean_layer = Lambda(lambda xin: mean(xin, axis=1), name="AMeanLayer")
tmean_layer = Lambda(lambda xin: mean(xin, axis=1), name="TAMeanLayer")

X_question = gru2(gru1(question_vectors_input))             # (None, 3000, 50) -> (None, 60, 50)
X_question = Multiply()([question_masks_input, X_question]) # (None, 60, 50)
X_question = mean_layer(X_question)                         # (None, 60, 50) -> (None, 50)

X_triplets = Dense(50, activation='relu', name="Dense50T")(triplets_input)    # (None,500,900) -> (None,500,50)
X_triplets = tgru2(tgru1(X_triplets))                                         # (None,500,50)  -> (None,50)
X_triplets = tmean_layer(X_triplets)                                          # (None, 500, 50) -> (None, 50)
X_concatenated = Concatenate()([X_question, X_triplets])                      # (None,100)
X_concatenated = Dense(64, activation='relu', name="Dense64")(X_concatenated) # (None,100) -> (None,64)        
Y_pred = Dense(1, activation='sigmoid', name="Dense1")(X_concatenated)        # (None,64)  -> sigmoid    

model = Model(inputs=[question_vectors_input, question_masks_input, triplets_input], outputs=Y_pred)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question_vectors_input (InputLa (None, 60, 300)      0                                            
__________________________________________________________________________________________________
triplets_input (InputLayer)     (None, 500, 900)     0                                            
__________________________________________________________________________________________________
Sentence_GRU1 (GRU)             (None, 60, 50)       52650       question_vectors_input[0][0]     
__________________________________________________________________________________________________
Dense50T (Dense)                (None, 500, 50)      45050       triplets_input[0][0]             
__________________________________________________________________________________________________
question_m

In [19]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit([question_vectors, question_masks, triplet_vectors], Y, epochs = 30, batch_size = 20)

In [26]:
file_name = "train_epoch_" + str(4) + ".save"
model.load_weights(file_name)


from sklearn.metrics import classification_report, confusion_matrix
def EvalModel( model ):
    file_list         = ['X:/cs230/proj0/ned-graphs/data_wiki_encoded/dev/dev_10.ipwk',
                         'X:/cs230/proj0/ned-graphs/data_wiki_encoded/dev/dev_11.ipwk',
                         'X:/cs230/proj0/ned-graphs/data_wiki_encoded/dev/dev_12.ipwk',
                         'X:/cs230/proj0/ned-graphs/data_wiki_encoded/dev/dev_13.ipwk'
                        ]
    
    file_list_count   = len(file_list)
    
    data = []
    file_index       = 0
    # train on batches of files
    for file_index in range(0, file_list_count) :
        inputFile = file_list[file_index]
        file_index       += 1
        # print(inputFile)
        f = open(inputFile, 'rb')
        test = pickle.load(f)
        data = np.append(data, test)
        f.close()
    
    question_vectors, question_masks, triplet_vectors, Y = prepare_data(data, 
                                                                                question_max_words   = s_QuestionMaxWords, 
                                                                                triplets_max_numbers = s_TripletsMaxNumbers, 
                                                                                dimOfQuestionVector  = s_DimOfQuestionVector, 
                                                                                dimOfTripletVector   = s_DimOfTripletVector, 
                                                                                dimOfMask            = s_DimOfMask )
        
    
    loss, acc = model.evaluate([question_vectors, question_masks, triplet_vectors], Y)
    print( "loss = " + str(loss) + " acc  " + str(acc))
    Y_pred = model.predict([question_vectors, question_masks, triplet_vectors])
    Y_pred = np.squeeze(Y_pred)
    #print(y.astype(np.float))
    print(confusion_matrix(Y, Y_pred>0.5))
    print(classification_report(Y, Y_pred>0.5))

EvalModel(model)   

MemoryError: 

In [10]:
#TRAIN
s_NumEpocs        = 5
s_NumFilesInBatch = 5
file_list         = glob.glob('X:/cs230/proj0/ned-graphs/data_wiki_encoded/train/train_*.ipwk')
file_list_count   = len(file_list)
file_batch_count  = math.ceil(( file_list_count / s_NumFilesInBatch ))

print(file_list_count)
print(file_batch_count)


459
92


In [11]:
for ep in range(2,s_NumEpocs):
    if ep != 0:
        file_name = "train_epoch_" + str(ep-1) + ".save"
        model.load_weights(file_name)
    
    file_index       = 0
    batch_file_index = 0
    
    data = []
    # train on batches of files
    for file_index in range(0, file_list_count) :
        inputFile = file_list[file_index]
        file_index       += 1
        batch_file_index += 1
        # print(inputFile)
        f = open(inputFile, 'rb')
        test = pickle.load(f)
        data = np.append(data, test)
        f.close()
    
        if batch_file_index == s_NumFilesInBatch or file_index == file_list_count:
            print("Training on Epoch ", str(ep), "batch ", str(batch_file_index), " Files [" + str(s_NumFilesInBatch) + "] start ", str(file_index - batch_file_index), "/", str(file_list_count) )
            question_vectors, question_masks, triplet_vectors, Y = prepare_data(data, 
                                                                                question_max_words   = s_QuestionMaxWords, 
                                                                                triplets_max_numbers = s_TripletsMaxNumbers, 
                                                                                dimOfQuestionVector  = s_DimOfQuestionVector, 
                                                                                dimOfTripletVector   = s_DimOfTripletVector, 
                                                                                dimOfMask            = s_DimOfMask )
            # train on this batch
            model.fit([question_vectors, question_masks, triplet_vectors], Y, epochs = 10, batch_size = 200, shuffle=True)
            # rest for next batch
            batch_file_index = 0;
            data = []
            
    # save the mode for the epoch
    file_name = "train_epoch_" + str(ep) + ".save"
    model.save_weights(file_name)
    



Training on Epoch  2 batch  5  Files [5] start  0 / 459
Epoch 1/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.2591 - acc: 0.9060
Epoch 2/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.1631 - acc: 0.9390
Epoch 3/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.1212 - acc: 0.9590
Epoch 4/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0990 - acc: 0.9660
Epoch 5/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0797 - acc: 0.9750
Epoch 6/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0683 - acc: 0.9770
Epoch 7/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0586 - acc: 0.9810
Epoch 8/10
1000/1000 [==============================] - 11s 11ms/step - loss: 0.0515 - acc: 0.9810
Epoch 9/10
1000/1000 [==============================] - 12s 12ms/step - loss: 0.0447 - acc: 0.9840
Epoch 10/10
1000/1000 [==============================

1000/1000 [==============================] - 22s 22ms/step - loss: 0.0622 - acc: 0.9710
Epoch 10/10
1000/1000 [==============================] - 22s 22ms/step - loss: 0.0575 - acc: 0.9770
Training on Epoch  2 batch  5  Files [5] start  40 / 459
Epoch 1/10
1000/1000 [==============================] - 26s 26ms/step - loss: 0.2884 - acc: 0.9090
Epoch 2/10
1000/1000 [==============================] - 27s 27ms/step - loss: 0.2026 - acc: 0.9260
Epoch 3/10
1000/1000 [==============================] - 27s 27ms/step - loss: 0.1449 - acc: 0.9490
Epoch 4/10
1000/1000 [==============================] - 27s 27ms/step - loss: 0.1137 - acc: 0.9620
Epoch 5/10
1000/1000 [==============================] - 27s 27ms/step - loss: 0.1021 - acc: 0.9700
Epoch 6/10
1000/1000 [==============================] - 28s 28ms/step - loss: 0.0875 - acc: 0.9750
Epoch 7/10
1000/1000 [==============================] - 27s 27ms/step - loss: 0.0777 - acc: 0.9780
Epoch 8/10
1000/1000 [==============================] - 27s 27

1000/1000 [==============================] - 48s 48ms/step - loss: 0.1800 - acc: 0.9250
Epoch 6/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1572 - acc: 0.9420
Epoch 7/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1440 - acc: 0.9450
Epoch 8/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1328 - acc: 0.9470
Epoch 9/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.1250 - acc: 0.9500
Epoch 10/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.1187 - acc: 0.9520
Training on Epoch  2 batch  5  Files [5] start  120 / 459
Epoch 1/10
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2589 - acc: 0.8770
Epoch 2/10
1000/1000 [==============================] - 39s 39ms/step - loss: 0.2247 - acc: 0.8920
Epoch 3/10
1000/1000 [==============================] - 40s 40ms/step - loss: 0.2062 - acc: 0.9060
Epoch 4/10
1000/1000 [==============================] - 41s 4

1000/1000 [==============================] - 37s 37ms/step - loss: 0.1212 - acc: 0.9450
Epoch 4/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.1011 - acc: 0.9520
Epoch 5/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0908 - acc: 0.9540
Epoch 6/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0845 - acc: 0.9590
Epoch 7/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0765 - acc: 0.9640
Epoch 8/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0718 - acc: 0.9680
Epoch 9/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0679 - acc: 0.9700
Epoch 10/10
1000/1000 [==============================] - 37s 37ms/step - loss: 0.0649 - acc: 0.9700
Training on Epoch  2 batch  5  Files [5] start  160 / 459
Epoch 1/10
1000/1000 [==============================] - 38s 38ms/step - loss: 0.2360 - acc: 0.9040
Epoch 2/10
1000/1000 [==============================] - 38s 3

1000/1000 [==============================] - 54s 54ms/step - loss: 0.2337 - acc: 0.8920
Epoch 2/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1918 - acc: 0.9030
Epoch 3/10
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1714 - acc: 0.9060
Epoch 4/10
1000/1000 [==============================] - 58s 58ms/step - loss: 0.1559 - acc: 0.9060
Epoch 5/10
1000/1000 [==============================] - 58s 58ms/step - loss: 0.1473 - acc: 0.9120
Epoch 6/10
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1396 - acc: 0.9210
Epoch 7/10
1000/1000 [==============================] - 57s 57ms/step - loss: 0.1340 - acc: 0.9250
Epoch 8/10
1000/1000 [==============================] - 57s 57ms/step - loss: 0.1283 - acc: 0.9220
Epoch 9/10
1000/1000 [==============================] - 57s 57ms/step - loss: 0.1250 - acc: 0.9310
Epoch 10/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1224 - acc: 0.9320
Training on Epoch  2

...

1000/1000 [==============================] - 78s 78ms/step - loss: 0.1020 - acc: 0.9460
Epoch 10/10
1000/1000 [==============================] - 78s 78ms/step - loss: 0.1001 - acc: 0.9470
Training on Epoch  4 batch  5  Files [5] start  290 / 459
Epoch 1/10
1000/1000 [==============================] - 77s 77ms/step - loss: 0.2145 - acc: 0.9150
Epoch 2/10
1000/1000 [==============================] - 76s 76ms/step - loss: 0.1532 - acc: 0.9200
Epoch 3/10
1000/1000 [==============================] - 76s 76ms/step - loss: 0.1296 - acc: 0.9340
Epoch 4/10
1000/1000 [==============================] - 76s 76ms/step - loss: 0.1131 - acc: 0.9410
Epoch 5/10
1000/1000 [==============================] - 76s 76ms/step - loss: 0.1055 - acc: 0.9460
Epoch 6/10
1000/1000 [==============================] - 76s 76ms/step - loss: 0.0967 - acc: 0.9480
Epoch 7/10
1000/1000 [==============================] - 76s 76ms/step - loss: 0.0921 - acc: 0.9520
Epoch 8/10
1000/1000 [==============================] - 76s 7

1000/1000 [==============================] - 68s 68ms/step - loss: 0.0557 - acc: 0.9710
Epoch 8/10
1000/1000 [==============================] - 69s 69ms/step - loss: 0.0532 - acc: 0.9740
Epoch 9/10
1000/1000 [==============================] - 68s 68ms/step - loss: 0.0513 - acc: 0.9750
Epoch 10/10
1000/1000 [==============================] - 68s 68ms/step - loss: 0.0491 - acc: 0.9740
Training on Epoch  4 batch  5  Files [5] start  330 / 459
Epoch 1/10
1000/1000 [==============================] - 67s 67ms/step - loss: 0.2207 - acc: 0.9080
Epoch 2/10
1000/1000 [==============================] - 65s 65ms/step - loss: 0.1613 - acc: 0.9290
Epoch 3/10
1000/1000 [==============================] - 66s 66ms/step - loss: 0.1223 - acc: 0.9400
Epoch 4/10
1000/1000 [==============================] - 66s 66ms/step - loss: 0.1036 - acc: 0.9570
Epoch 5/10
1000/1000 [==============================] - 66s 66ms/step - loss: 0.0923 - acc: 0.9580
Epoch 6/10
1000/1000 [==============================] - 66s 6

1000/1000 [==============================] - 66s 66ms/step - loss: 0.0904 - acc: 0.9700
Epoch 6/10
1000/1000 [==============================] - 66s 66ms/step - loss: 0.0763 - acc: 0.9770
Epoch 7/10
1000/1000 [==============================] - 67s 67ms/step - loss: 0.0635 - acc: 0.9840
Epoch 8/10
1000/1000 [==============================] - 67s 67ms/step - loss: 0.0547 - acc: 0.9840
Epoch 9/10
1000/1000 [==============================] - 67s 67ms/step - loss: 0.0476 - acc: 0.9880
Epoch 10/10
1000/1000 [==============================] - 66s 66ms/step - loss: 0.0536 - acc: 0.9840
Training on Epoch  4 batch  5  Files [5] start  370 / 459
Epoch 1/10
1000/1000 [==============================] - 78s 78ms/step - loss: 0.3319 - acc: 0.8980
Epoch 2/10
1000/1000 [==============================] - 79s 79ms/step - loss: 0.2384 - acc: 0.9130
Epoch 3/10
1000/1000 [==============================] - 78s 78ms/step - loss: 0.1693 - acc: 0.9350
Epoch 4/10
1000/1000 [==============================] - 81s 8

1000/1000 [==============================] - 73s 73ms/step - loss: 0.1375 - acc: 0.9470
Epoch 4/10
1000/1000 [==============================] - 74s 74ms/step - loss: 0.1161 - acc: 0.9550
Epoch 5/10
1000/1000 [==============================] - 73s 73ms/step - loss: 0.0938 - acc: 0.9660
Epoch 6/10
1000/1000 [==============================] - 73s 73ms/step - loss: 0.0795 - acc: 0.9720
Epoch 7/10
1000/1000 [==============================] - 74s 74ms/step - loss: 0.0666 - acc: 0.9780
Epoch 8/10
1000/1000 [==============================] - 74s 74ms/step - loss: 0.0583 - acc: 0.9820
Epoch 9/10
1000/1000 [==============================] - 72s 72ms/step - loss: 0.0520 - acc: 0.9830
Epoch 10/10
1000/1000 [==============================] - 73s 73ms/step - loss: 0.0464 - acc: 0.9870
Training on Epoch  4 batch  5  Files [5] start  410 / 459
Epoch 1/10
1000/1000 [==============================] - 59s 59ms/step - loss: 0.2817 - acc: 0.8960
Epoch 2/10
1000/1000 [==============================] - 58s 5

1000/1000 [==============================] - 61s 61ms/step - loss: 0.1969 - acc: 0.9370
Epoch 2/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1300 - acc: 0.9540
Epoch 3/10
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0890 - acc: 0.9680
Epoch 4/10
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0694 - acc: 0.9720
Epoch 5/10
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0544 - acc: 0.9810
Epoch 6/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0436 - acc: 0.9840
Epoch 7/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0360 - acc: 0.9890
Epoch 8/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0310 - acc: 0.9910
Epoch 9/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.0280 - acc: 0.9930
Epoch 10/10
1000/1000 [==============================] - 61s 61ms/step - loss: 0.0249 - acc: 0.9930
Training on Epoch  4

In [ ]:
# from keras.layers import Dense, Input, Dropout, LSTM, Activation, SimpleRNN, GRU, Concatenate, Multiply, Reshape, Flatten, Bidirectional

# # layers
# s1_bilstm = Bidirectional(GRU(128, return_sequences = True), name = "s1_bilstm")
# s2_bilstm = Bidirectional(GRU(128, return_sequences = False), name = "s2_bilstm")
# t_bilstm = Bidirectional(GRU(128, return_sequences = False), name = "t_bilstm")
# embedding_layer = pretrained_embedding_layer(word_to_vec_map, words_to_index)
# unstack_layer = Lambda(lambda xin: myUnstack(xin), name="AUnstackLayer")

# # sentence inputs
# sentence_indices = Input(shape=(maxWordsPerSentence,), dtype='int32', name="sentence_indices")   
# sentence_masks = Input(shape=(2*lstm_dim,), dtype='float32', name="sentence_masks")

# # sentence embeddings
# sentence_embeddings = embedding_layer(sentence_indices)       
# # X_sentence = Reshape((maxWordsPerSentence, -1), name="s_reshape")(sentence_embeddings)
# X_sentence = s1_bilstm(sentence_embeddings)
# X_sentence = Multiply()([sentence_masks, X_sentence]) # (None, 60, 256)
# X_sentence = s2_bilstm(X_sentence)

# # triplets
# triplets_input = Input(shape=(maxTriplets, maxWordsPerSentence,), dtype='int32', name="tripletz") 
# triplet_embeddings = embedding_layer(triplets_input)

# X_triplets = Reshape((maxTriplets, -1))(triplet_embeddings)
# X_triplets = t_bilstm(X_triplets)

# X_concatenated = Concatenate()([X_sentence, X_triplets])
# X_concatenated = Dense(256, activation='relu', name="Dense256")(X_concatenated)
# X_concatenated = Dense(64, activation='relu', name="Dense64")(X_concatenated)
# Y_pred = Dense(1, activation='sigmoid', name="Dense1")(X_concatenated)

# m.v2 = Model(inputs=[sentence_indices, sentence_masks, triplets_input], outputs=X_concatenated)
# print(m.v2.summary())